In [1]:
from google.colab import drive
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob
from sklearn.preprocessing import normalize
import gc
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, GlobalMaxPooling2D 
from keras.models import Model
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras.models import load_model
from keras.layers import Dropout
from keras import backend as K
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
drive.mount('/content/drive')
root_path = 'drive/My Drive/Colab Notebooks/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [0]:
#load the training data of one single angle of the exams (axial | coronal | sagittal) by giving the angle name
def trainingSet(file_name):
    training_set = []
    for np_name in sorted(glob.glob('MRNet-v1.0/train/'+file_name+'/*.npy')):
        training_set.append(np.load(np_name))
    return training_set  

In [0]:
#load the labels of one knee tear(abnormal | acl | meniscus) by the giving the file name that contains the labels
def load_labels(train_file_name):
    training_labels = np.loadtxt(train_file_name, delimiter=',')[:,1]
    return training_labels

In [0]:
#as each exam (stack) contains S slices and most of the pretrained CNN models accept the image with channel = 3
#extract the middle 3 slices as most of the time they deside if the patiant has a certain tear or not
def extract_3(dataset, data_size):
    for i in range(len(dataset)):
        dataset[i] = list(dataset[i])
        middel_idx = int(len(dataset[i]) / 2)
        dataset[i] = dataset[i][middel_idx - 1 : middel_idx + 2]
        dataset[i] = np.array(dataset[i])
    dataset = np.array(dataset)
    dataset = dataset.reshape(data_size, 256, 256 ,3)
    return dataset

In [0]:
#load the training data for each angle of the knee
training_set_axial= trainingSet('axial')
training_set_coronal=trainingSet('coronal')
training_set_sagittal=trainingSet('sagittal')

In [0]:
#extract the middel 3 slices from each stack
training_set_axial = extract_3(training_set_axial, 1130)
training_set_coronal =extract_3(training_set_coronal, 1130)
training_set_sagittal =extract_3(training_set_sagittal, 1130)

In [9]:
training_set_axial.shape

(1130, 256, 256, 3)

In [0]:
#load the labels of the trainig data for each knee tear 
training_label_abnormal = load_labels('MRNet-v1.0/train-abnormal.csv')
training_label_acl = load_labels('MRNet-v1.0/train-acl.csv')
training_label_meniscus = load_labels('MRNet-v1.0/train-meniscus.csv')

In [11]:
training_label_abnormal.shape

(1130,)

In [0]:
datagen = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.05,
        height_shift_range=0.05,
        zoom_range=0.01,
        fill_mode='nearest')

In [0]:
def data_augmentation(dataset, labels):
  augmented_data = []
  augmented_labels = []
  num_augmented = 0
  for X_batch, y_batch in datagen.flow(dataset, labels, batch_size= dataset.shape[0], shuffle=False):
      augmented_data.append(X_batch)
      augmented_labels.append(y_batch)
      num_augmented += dataset.shape[0]
      if num_augmented == dataset.shape[0] * 2:
          break
          
  dataset = np.array(augmented_data).reshape(num_augmented, 256, 256, 3)
  labels = np.array(augmented_labels).reshape(num_augmented, 1)
  return dataset, labels

In [0]:
training_set_axial, training_label_abnormal = data_augmentation(training_set_axial, training_label_abnormal)

In [0]:
training_set_coronal, training_label_acl = data_augmentation(training_set_coronal, training_label_acl)

In [0]:
training_set_sagittal, training_label_meniscus = data_augmentation(training_set_sagittal, training_label_meniscus)

In [0]:
training_set_axial.shape

In [0]:
#load INCEPTIONV3 pretrainned model from keras without the top layer &
#specify the shape of the input image as (256x256x3) 
mri_InceptionV3 = InceptionV3(include_top = False, input_shape = (256, 256, 3))

#remove the last one
mri_InceptionV3.layers.pop()

#get the output of the INCEPTIONV# model "feature extraction" and add Dense layers so that the model is designed
#to classify our data
mri_InceptionV3_output = mri_InceptionV3.output

mri_InceptionV3_last_layer = GlobalMaxPooling2D()(mri_InceptionV3_output)

mri_InceptionV3_last_layer = Dense(1024, activation = 'relu')(mri_InceptionV3_last_layer)

mri_InceptionV3_last_layer = Dense(128, activation = 'relu')(mri_InceptionV3_last_layer)

mri_InceptionV3_last_layer = Dense(32, activation = 'relu')(mri_InceptionV3_last_layer)

predictor = Dense(1, activation='sigmoid')(mri_InceptionV3_last_layer)

modified_mri_InceptionV3 = Model(inputs = mri_InceptionV3.input, output = predictor)

modified_mri_InceptionV3.summary()

In [0]:
#freeze the added dense layers and unfreeze the rest:
for layer in modified_mri_InceptionV3.layers[15:]:
    layer.trainable = True 
    
for layer in mri_InceptionV3.layers:
    layer.trainable = False
    
modified_mri_InceptionV3.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

In [0]:
class Metrics(Callback):
  def on_train_begin(self, logs={}):
    self.val_f1s = []
  def on_epoch_end(self, epoch, logs={}):
    val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
    val_targ = self.validation_data[1]
    _val_f1 = f1_score(val_targ, val_predict)
    self.val_f1s.append(_val_f1)
    print ('-val_f1 : %f '%(_val_f1))
    return
metrics = Metrics()

In [0]:
def plot_history(history):
  plt.figure(figsize=(8, 6))
  ax = plt.gca()
  plt.title('Training performance')
  plt.plot(history.epoch, history.history['loss'], label='Loss')
  plt.plot(history.epoch, history.history['val_loss'], label='validation error')
  plt.plot(history.epoch, history.history['acc'], label='accuracy')
  plt.plot(history.epoch, history.history['val_acc'], label='validation accuracy')
  plt.plot(metrics.val_f1s, label='f_score')
  plt.grid(True,color='white')
  mpl.style.use('seaborn')
  plt.legend() 
  

In [0]:
#train the model with 1073 trainning data & 57 validation data
#save the model that has the max validation accuracy
#at the end plot the trainng loss & accuracy, validation loss & accuracy & F-score 
def fit_model(training_set, training_labels, model_file_name):
  
    history = modified_mri_InceptionV3.fit(x = training_set, y = training_labels, batch_size=32,
                       epochs=30, validation_split = 0.05,
                       callbacks =[
                          EarlyStopping(patience=3, monitor = 'val_loss', mode = 'min'),
                          ModelCheckpoint(model_file_name, monitor='val_acc', save_best_only=True, mode='max', period=1), metrics])
    plot_history(history)
    best_model = load_model(model_file_name)
    return best_model

In [0]:
#train the modified INCEPTIONV3 model that will accept the axial data and classify the knee is it has an abnormal tear or not 
axial_abnormal_model = fit_model(training_set_axial, training_label_abnormal, "axial_abnormal_model")

In [0]:
#train the modified INCEPTIONV3 model that will accept the axial data and classify the knee is it has an acl tear or not 
axial_acl_model = fit_model(training_set_axial, training_label_acl, "axial_acl_model")

In [0]:
#train the modified INCEPTIONV3 model that will accept the axial data and classify the knee is it has a meniscus tear or not 
axial_meniscus_model = fit_model(training_set_axial, training_label_meniscus, "axial_meniscus_model")

In [0]:
#train the modified INCEPTIONV3 model that will accept the coronal data and classify the knee is it has an abnormal tear or not 
coronal_abnormal_model = fit_model(training_set_coronal, training_label_abnormal, "coronal_abnormal_model")

In [0]:
#train the modified INCEPTIONV3 model that will accept the coronal data and classify the knee is it has an acl tear or not 
coronal_acl_model = fit_model(training_set_coronal, training_label_acl, "coronal_acl_model")

In [0]:
#train the modified INCEPTIONV3 model that will accept the coronal data and classify the knee is it has a meniscus tear or not 
coronal_meniscus_model = fit_model(training_set_coronal, training_label_meniscus, "coronal_meniscus_model")

In [0]:
#train the modified INCEPTIONV3 model that will accept the sagittal data and classify the knee is it has an abnormal tear or not 
sagittal_abnormal_model = fit_model(training_set_sagittal, training_label_abnormal, "sagittal_abormal_model")

In [0]:
#train the modified INCEPTIONV3 model that will accept the sagittal data and classify the knee is it has an acl tear or not 
sagittal_acl_model = fit_model(training_set_sagittal, training_label_acl, "sagittal_acl_model")

In [0]:
#train the modified INCEPTIONV3 model that will accept the sagittal data and classify the knee is it has a meniscus tear or not 
sagittal_meniscus_model = fit_model(training_set_sagittal, training_label_meniscus, "sagittal_meniscus_model")

## Evaluate the 9 models

In [0]:
#load the test data of one single angle of the exams (axial | coronal | sagittal) by giving the angle name
def testSet(file_name):
    test_set = []
    for np_name in sorted(glob.glob('MRNet-v1.0/valid/'+file_name+'/*.npy')):
        test_set.append(np.load(np_name))
    return test_set   

In [0]:
#load the test data for each angle of the knee
test_set_axial= testSet('axial')
test_set_coronal=testSet('coronal')
test_set_sagittal=testSet('sagittal')

In [0]:
#extract the middel 3 slices from each stack
test_set_axial = extract_3(test_set_axial, 120)
test_set_coronal =extract_3(test_set_coronal, 120)
test_set_sagittal =extract_3(test_set_sagittal, 120)

In [0]:
#load the labels of the test data for each knee tear 
test_label_abnormal = load_labels('MRNet-v1.0/valid-abnormal.csv')
test_label_acl = load_labels('MRNet-v1.0/valid-acl.csv')
test_label_meniscus = load_labels('MRNet-v1.0/valid-meniscus.csv')

In [0]:
axial_abnormal_model.evaluate(test_set_axial, test_label_abnormal)

In [0]:
axial_acl_model.evaluate(test_set_axial, test_label_acl) 

In [0]:
axial_meniscus_model.evaluate(test_set_axial, test_label_meniscus) 

In [0]:
coronal_abnormal_model.evaluate(test_set_coronal, test_label_abnormal)

In [0]:
coronal_acl_model.evaluate(test_set_coronal, test_label_acl)

In [0]:
coronal_meniscus_model.evaluate(test_set_coronal, test_label_meniscus)

In [0]:
sagittal_abnormal_model.evaluate(test_set_sagittal, test_label_abnormal)

In [0]:
sagittal_acl_model.evaluate(test_set_sagittal, test_label_acl)

In [0]:
sagittal_meniscus_model.evaluate(test_set_sagittal, test_label_meniscus)

In [0]:
#predict if the knee has an acl tear by using the data of the 3 knee angel (axial, coronal & sagittal)
#and the 3 models that accept one of these data and predict if the knee has an abnromal tear or not
#by doing a majority voting between the 3 models
def predict_abnormal(axial_input, coronal_input, sagittal_input, ground_truth):
    axial_abnormal_prediction = axial_abnormal_model.predict(axial_input)
    coronal_abnormal_prediction = coronal_abnormal_model.predict(coronal_input)
    sagittal_abnormal_prediction = sagittal_abnormal_model.predict(sagittal_input)
    
    #to say that the exam has the acl tear or not, each of the models will vote in this decision
    #and the final decision will be made by taking the max vote
    predict_voting = []
    for i in range (len(axial_abnormal_prediction)):
        voting_list =[0, 0] 
        
        #make the axial_acl_model vote for the final decision 
        if(axial_abnormal_prediction[i]>0.5):
            voting_list[1] +=1
        else:
            voting_list[0] += 1
    
        #make the coronal_acl_model vote for the final decision 
        if(coronal_abnormal_prediction[i]>0.5):
            voting_list[1] +=1
        else:
            voting_list[0] += 1
            
        #make the sagittal_acl_model vote for the final decision 
        if(sagittal_abnormal_prediction[i]>0.5):
            voting_list[1] +=1
        else:
            voting_list[0] += 1
        
        #give the exam the prediction that has the most votes
        predict_voting.append(np.argmax(voting_list))
    
  
    return accuracy_score(ground_truth, np.array(predict_voting)) * 100

In [0]:
#predict if the knee has an acl tear by using the data of the 3 knee angel (axial, coronal & sagittal)
#and the 3 models that accept one of these data and predict if the knee has an acl tear or not
#by doing a majority voting between the 3 models
def predict_acl(axial_input, coronal_input, sagittal_input, ground_truth):
    axial_acl_prediction = axial_acl_model.predict(axial_input)
    coronal_acl_prediction = coronal_acl_model.predict(coronal_input)
    sagittal_acl_prediction = sagittal_acl_model.predict(sagittal_input)
    
    #to say that the exam has the acl tear or not, each of the models will vote in this decision
    #and the final decision will be made by taking the max vote
    predict_voting = []
    for i in range (len(axial_acl_prediction)):
        voting_list =[0, 0] 
        
        #make the axial_acl_model vote for the final decision 
        if(axial_acl_prediction[i]>0.5):
            voting_list[1] +=1
        else:
            voting_list[0] += 1
    
        #make the coronal_acl_model vote for the final decision 
        if(coronal_acl_prediction[i]>0.5):
            voting_list[1] +=1
        else:
            voting_list[0] += 1
            
        #make the sagittal_acl_model vote for the final decision 
        if(sagittal_acl_prediction[i]>0.5):
            voting_list[1] +=1
        else:
            voting_list[0] += 1
        
        #give the exam the prediction that has the most votes
        predict_voting.append(np.argmax(voting_list))
    
  
    return accuracy_score(ground_truth, np.array(predict_voting)) * 100

In [0]:
#predict if the knee has an meniscus tear by using the data of the 3 knee angel (axial, coronal & sagittal)
#and the 3 models that accept one of these data and predict if the knee has an meniscus tear or not
#by doing a majority voting between the 3 models
def predict_meniscus(axial_input, coronal_input, sagittal_input, ground_truth):
    axial_meniscus_prediction = axial_meniscus_model.predict(axial_input)
    coronal_meniscus_prediction = coronal_meniscus_model.predict(coronal_input)
    sagittal_meniscus_prediction = sagittal_meniscus_model.predict(sagittal_input)
    
    #to say that the exam has the meniscus tear or not, each of the models will vote in this decision
    #and the final decision will be made by taking the max vote
    predict_voting = []
    for i in range (len(axial_meniscus_prediction)):
        voting_list =[0, 0] 
        
        #make the axial_meniscus_model vote for the final decision 
        if(axial_meniscus_prediction[i]>0.5):
            voting_list[1] +=1
        else:
            voting_list[0] += 1
    
        #make the coronal_meniscus_model vote for the final decision 
        if(coronal_meniscus_prediction[i]>0.5):
            voting_list[1] +=1
        else:
            voting_list[0] += 1
            
        #make the sagittal_meniscus_model vote for the final decision 
        if(sagittal_meniscus_prediction[i]>0.5):
            voting_list[1] +=1
        else:
            voting_list[0] += 1
        
        #give the exam the prediction that has the most votes
        predict_voting.append(np.argmax(voting_list))
    
  
    return accuracy_score(ground_truth, np.array(predict_voting)) * 100

In [0]:
#evaluate the abnormal ensemble models
print (predict_abnormal(test_set_axial, test_set_coronal, test_set_sagittal, test_label_abnormal))

In [0]:
#evaluate the acl ensemble models
print (predict_acl(test_set_axial, test_set_coronal, test_set_sagittal, test_label_acl))

In [0]:
#evaluate the meniscus ensemble models
print (predict_meniscus(test_set_axial, test_set_coronal, test_set_sagittal, test_label_meniscus))